In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Conv2D
import h5py
from rainnet.rainnet import rainnet
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
# from memory_profiler import profile

In [2]:
#@title Building the RainNet model and load its pretrained weights

base_model = rainnet(input_shape=(640,800,4))
base_model.load_weights("weights/rainnet_weights.h5")

In [3]:
h5= h5py.File('data/20170604.h5')

/home/allen/Documents/Programs/miniconda3/envs/dl/lib/python3.7/site-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """Entry point for launching an IPython kernel.


In [4]:
x_train= np.stack([h5['train-%d'%i][:].transpose(1,2,0) for i in range(60)])
y_train= np.stack([h5['test-%d'%i][:].transpose(1,2,0) for i in range(60)])

In [5]:
h5.close()

In [6]:
base_model.layers[-2]

In [7]:
for i in range(len(base_model.layers[:-2])):
    base_model.layers[i].trainable = False
# inputs= keras.Input(shape=(640,800,4))
# x= base_model(inputs, training=False)
# x= Conv2D(16,3, padding='same', kernel_initializer='he_normal')(x)
# x= Activation("relu")(x)
# x= Conv2D(1,, padding='same', kernel_initializer='he_normal')(x)
# x= Activation("relu")(x)
# outputs= keras.layers.Conv2D(1,1, activation='linear')(x)
# model= keras.Model(inputs, outputs)



In [8]:
base_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 640, 800, 4) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 640, 800, 64) 2368        input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 640, 800, 64) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 640, 800, 64) 36928       activation[0][0]                 
_______________________________________________________________________________________

model.compile(
    optimizer=keras.optimizers.Nadam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.MeanSquaredError(),
    # List of metrics to monitor
    metrics=[keras.metrics.MeanSquaredError()],
)

In [9]:
loss_fn= keras.losses.MeanSquaredError()
optimizer= keras.optimizers.Adam()

epoches=20
batches=1
bsize=len(x_train)//batches

for e in range(epoches):
#     for i, (input, target) in enumerate(zip(x_train, y_train)):
    b=0
    while b<bsize:

        inputs= x_train[b*batches:(b+1)*batches,:,:,:]
        targets= y_train[b*batches:(b+1)*batches,:,:,:]
        with tf.GradientTape() as tape:
            prediction= base_model(inputs)
            loss_value= loss_fn(targets, prediction)
        gradients= tape.gradient(loss_value, base_model.trainable_weights)
        optimizer.apply_gradients(zip(gradients, base_model.trainable_weights))

        print('[%d/%d][%d/%d] loss: %.4f'%(e, epoches, b, bsize, loss_value))
        b+=1


[0/20][0/60] loss: 0.0066
[0/20][1/60] loss: 0.0288
[0/20][2/60] loss: 0.0934
[0/20][3/60] loss: 0.1375
[0/20][4/60] loss: 0.2386
[0/20][5/60] loss: 1.1969
[0/20][6/60] loss: 1.7778
[0/20][7/60] loss: 4.4480
[0/20][8/60] loss: 6.8218
[0/20][9/60] loss: 5.9605
[0/20][10/60] loss: 8.0087
[0/20][11/60] loss: 8.3426
[0/20][12/60] loss: 6.0098
[0/20][13/60] loss: 12.9322
[0/20][14/60] loss: 16.9777
[0/20][15/60] loss: 14.7470
[0/20][16/60] loss: 21.9815
[0/20][17/60] loss: 4.7834
[0/20][18/60] loss: 14.5361
[0/20][19/60] loss: 45.0854
[0/20][20/60] loss: 60.9113


KeyboardInterrupt: 

model.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.MeanSquaredError(),
    # List of metrics to monitor
    metrics=[keras.metrics.MeanSquaredError()],
)

epochs = 20
model.fit(x_train, y_train, epochs=epochs)

In [ ]:
h5= h5py.File('data/20170604.h5')

In [ ]:
inputs= h5['train-1'][:]
ref= h5['test-1'][:]
h5.close()

In [ ]:
results= model.predict(inputs.transpose(1,2,0)[np.newaxis,:,:,:4])

In [ ]:
results[results<0]=0
plt.imshow(results.squeeze())
plt.colorbar()

In [ ]:
fig=plt.figure(figsize=(10,5))
ax= fig.add_subplot(121)
plt.imshow(results.squeeze())
ax=fig.add_subplot(122)
plt.imshow(ref[0].astype(np.float32))

In [ ]:
fig=plt.figure()
plt.imshow(inputs[0,:,:].astype(np.float32))
fig=plt.figure()
# plt.colorbar()
plt.imshow(inputs[1,:,:].astype(np.float32))
fig=plt.figure()
# plt.colorbar()
plt.imshow(inputs[2,:,:].astype(np.float32))
fig=plt.figure()
# plt.colorbar()
plt.imshow(inputs[3,:,:].astype(np.float32))
fig=plt.figure()
# plt.colorbar()
plt.imshow(ref[0,:,:].astype(np.float32))
# plt.colorbar()

In [ ]:
plt.imshow()